In [81]:
import pandas as pd
import argparse as ap
import datetime as dt
import numpy as np
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

def to_img(arr):
    return Image.fromarray(np.uint8(arr))

def update_image(image):
    fig, ax = plt.subplots()
    ax.imshow(image)
    ax.axis('off')
    plt.show()

In [82]:
uploaded_file = "./dog_small.png"
img1 = Image.open(uploaded_file) if uploaded_file else None

second_uploaded_file = "./parrot_small.png"
img2 = Image.open(second_uploaded_file) if second_uploaded_file else None

In [83]:

img1_plot = widgets.interactive(update_image, image = widgets.fixed(img1))
img2_plot = widgets.interactive(update_image, image = widgets.fixed(img2))
display(img1_plot,img2_plot)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [84]:
parameter = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Lambda:',
    continuous_update=False
)
display(parameter)

FloatSlider(value=1.0, continuous_update=False, description='Lambda:', max=1.0, step=0.01)

In [94]:
def mix_arrays_with_fft(arr1, arr2, parameter = 1):
    """
    Interpolates the phases of two arrays after applying the discrete Fourier transform and then applies them to the strength of the second array.
    parameter = 1 => phases only from the first array
    parameter = 0 => phases only from the second array (nothing will be changed)
    """
    fouriered1 = np.fft.fft2(arr1)
    angles1 = np.angle(fouriered1)
    strengths1 = np.abs(fouriered1)

    fouriered2 = np.fft.fft2(arr2)
    angles2 = np.angle(fouriered2)
    strengths2 = np.abs(fouriered2)

    angle_and_radius_mix = np.zeros(fouriered2.shape, dtype=complex)
    for x in np.arange(fouriered1.shape[0]):
        for y in np.arange(fouriered1.shape[1]):
            angle = angles1[x,y]*parameter + angles2[x,y]*(1-parameter)
            angle_and_radius_mix[x,y] = np.cos(angle) + np.sin(angle)*1j
            angle_and_radius_mix[x,y] *= strengths2[x,y]

    mixed = np.fft.ifft2(angle_and_radius_mix)
    mixed = np.abs(mixed)
    return mixed

# Transformations of the first image
def mix_images_with_fft_grayscale(img1, img2, parameter = 1):
    """
    Interpolates the phases of two images after applying the discrete Fourier transform and then applies them to the strength of the second image.
    parameter = 1 => phases only from the first image
    parameter = 0 => phases only from the second image (nothing will be changed)
    """
    array1 = np.asarray(img1)
    black_and_white1 = np.round(0.3*array1[:,:,0] + 0.6*array1[:,:,1] + 0.11*array1[:,:,2])
    
    # Transformations of the second image
    array2 = np.asarray(img2)
    black_and_white2 = np.round(0.3*array2[:,:,0] + 0.6*array2[:,:,1] + 0.11*array2[:,:,2])
   
    return mix_arrays_with_fft(black_and_white1, black_and_white2, parameter)


img_mixed = mix_images_with_fft_grayscale(img1, img2, parameter.value)
img_mixed_plot = widgets.interactive(update_image, image = widgets.fixed(img_mixed))


img_mixed_2 = mix_images_with_fft_grayscale(img2, img1, parameter.value)
img_mixed_plot_2 = widgets.interactive(update_image, image = widgets.fixed(img_mixed_2))

hbox = widgets.HBox([img1_plot, img2_plot])
display(hbox)
# Swapped to be in same order as in the paper
hbox_mixed = widgets.HBox([img_mixed_plot_2,img_mixed_plot])
display(hbox_mixed)



In [95]:
# Transformations of the first image
def mix_images_with_fft_rgb(img1, img2, parameter = 1):
    """
    Interpolates the phases of two images after applying the discrete Fourier transform and then applies them to the strength of the second image.
    parameter = 1 => phases only from the first image
    parameter = 0 => phases only from the second image (nothing will be changed)
    """
    array1 = np.asarray(img1)
    array2 = np.asarray(img2)
    
    r_mixed = mix_arrays_with_fft(array1[:,:,0], array2[:,:,0], parameter)
    g_mixed = mix_arrays_with_fft(array1[:,:,1], array2[:,:,1], parameter)
    b_mixed = mix_arrays_with_fft(array1[:,:,2], array2[:,:,2], parameter)
    return Image.fromarray(np.uint8(np.stack([r_mixed, g_mixed, b_mixed], axis=2)))


img_mixed_rgb = mix_images_with_fft_rgb(img1, img2, parameter.value)
img_mixed_rgb_plot = widgets.interactive(update_image, image = widgets.fixed(img_mixed_rgb))


img_mixed_rgb_2 = mix_images_with_fft_rgb(img2, img1, parameter.value)
img_mixed_rgb_plot_2 = widgets.interactive(update_image, image = widgets.fixed(img_mixed_rgb_2))

hbox = widgets.HBox([img1_plot, img2_plot])
display(hbox)
# Swapped to be in same order as in the paper
hbox_mixed = widgets.HBox([img_mixed_rgb_plot_2,img_mixed_rgb_plot])
display(hbox_mixed)

